# Reducción de la Dimensionalidad

## PCA

Para encontrar los componentes principales utilizamos una técnica estándar de factorización de matrices llamada *Descomposición en Valores Singulares* (SVD) que puede descomponer la matriz de entrenamiento $\bf{X}$ en la multiplicación de tres matrices $\bf{U}$ $\bf{\Sigma}$ $\bf{V}^T$, donde $\bf{V}$ contiene los vectores unitarios que definen todos los componentes principales.

$$
  \mathbf{V} = \left( 
    \begin{array}{cccc}
    | & | & & | \\
    c_1 & c_2 & ... & c_n \\
    | & | & & | 
    \end{array} \right)
$$

Una vez que has identificado todos los componentes principales, puedes reducir la dimensionalidad del conjunto de datos a $d$ dimensiones proyectándolo sobre el hiperplano definido por los primeros $d$ componentes principales.

$$
  \mathbf{X}_d = \mathbf{X} \mathbf{W}_d
$$

donde $\mathbf{W}_d$ contiene las primeras $d$ columnas de $\bf{V}$. Puedes recuperar el conjunto de datos original con

$$
  \mathbf{X} = \mathbf{X}_d \mathbf{W}_d^T
$$

Aunque se pierde algo de información debido a la proyección.

In [ ]:
# ============================================
# GENERACIÓN DE DATOS SINTÉTICOS EN 3D
# ============================================
# Este código crea un conjunto de datos 3D artificial para demostrar PCA

import numpy as np

# Fijamos la semilla para reproducibilidad de los resultados
np.random.seed(4)

# Configuración de parámetros
m = 60  # número de muestras (puntos) que vamos a generar
w1, w2 = 0.1, 0.3  # pesos para crear la tercera dimensión como combinación lineal
noise = 0.1  # nivel de ruido a añadir a los datos

# Generamos ángulos aleatorios para crear un patrón curvo
angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5

# Creamos matriz vacía de 60 filas x 3 columnas
X = np.empty((m, 3))

# Primera dimensión: combinación de cos y sin con ruido
X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2

# Segunda dimensión: función seno escalada con ruido
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2

# Tercera dimensión: COMBINACIÓN LINEAL de las dos primeras (baja varianza)
# Esta dimensión es redundante, por eso PCA la descartará
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)

In [ ]:
# ============================================
# VISUALIZACIÓN DE LOS DATOS EN 3D
# ============================================
# Graficamos los datos para ver su estructura en el espacio tridimensional

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

# Creamos figura con tamaño personalizado
fig = plt.figure(figsize=(6, 3.8))

# Añadimos subplot con proyección 3D
ax = fig.add_subplot(111, projection='3d')

# Graficamos los puntos: "bo" = blue circles (círculos azules)
ax.plot(X[:, 0], X[:, 1], X[:, 2], "bo")

# Configuramos los límites de cada eje para mejor visualización
ax.set_xlim([-1.5, 1.3])
ax.set_ylim([-1.2, 1.2])
ax.set_zlim([-1, 1])

plt.show()

In [ ]:
# ============================================
# DESCOMPOSICIÓN EN VALORES SINGULARES (SVD)
# ============================================
# SVD es la técnica matemática subyacente de PCA

# PASO 1: Centrar los datos (restar la media de cada columna)
# Esto es CRUCIAL para PCA: los datos deben tener media 0
X_centered = X - X.mean(axis=0)

# PASO 2: Aplicar SVD para descomponer la matriz en U, s, Vt
# X_centered = U @ S @ Vt
# - U: matriz de vectores singulares izquierdos (m x m)
# - s: valores singulares (diagonal de S)
# - Vt: matriz de vectores singulares derechos TRANSPUESTA (n x n)
#       Vt contiene los componentes principales en sus FILAS
U, s, Vt = np.linalg.svd(X_centered)

# Mostramos Vt: cada FILA es un componente principal
Vt

In [ ]:
# ============================================
# EXTRACCIÓN DE LOS PRIMEROS 2 COMPONENTES PRINCIPALES
# ============================================

# Los componentes principales están en las COLUMNAS de Vt.T (o FILAS de Vt)
# c1 es el primer componente principal (dirección de máxima varianza)
c1 = Vt.T[:, 0]

# c2 es el segundo componente principal (dirección de segunda mayor varianza)
# Es perpendicular (ortogonal) a c1
c2 = Vt.T[:, 1]

In [ ]:
# ============================================
# VERIFICACIÓN DE LA DESCOMPOSICIÓN SVD
# ============================================
# Comprobamos que U @ S @ Vt reconstruye X_centered

m, n = X.shape

# Creamos la matriz S completa (diagonal con valores singulares)
S = np.zeros(X_centered.shape)  # matriz de ceros del mismo tamaño que X_centered
S[:n, :n] = np.diag(s)  # colocamos los valores singulares en la diagonal

# Verificamos que U @ S @ Vt ≈ X_centered
# np.allclose verifica igualdad con tolerancia numérica
np.allclose(X_centered, U.dot(S).dot(Vt))

![](https://upload.wikimedia.org/wikipedia/commons/e/e9/Singular_value_decomposition.gif)

In [ ]:
# ============================================
# PROYECCIÓN MANUAL A 2D USANDO LOS PRIMEROS 2 COMPONENTES
# ============================================
# Reducimos de 3D a 2D proyectando sobre los 2 componentes principales

# W2 es la matriz de proyección: contiene los 2 primeros componentes principales
# Forma: (3, 2) - cada columna es un componente principal
W2 = Vt.T[:, :2]

# Proyectamos los datos centrados multiplicando por W2
# X_centered: (60, 3) @ W2: (3, 2) = X2D: (60, 2)
# Esto transforma cada punto 3D en un punto 2D
X2D = X_centered.dot(W2)

In [ ]:
# ============================================
# VISUALIZACIÓN DE LOS DATOS PROYECTADOS EN 2D
# ============================================
# Mostramos cómo se ven los datos después de la reducción dimensional

fig = plt.figure()
ax = fig.add_subplot(111, aspect='equal')  # aspect='equal' para mantener proporciones

# Graficamos los puntos en 2D con + y puntos
ax.plot(X2D[:, 0], X2D[:, 1], "k+")  # "k+" = cruces negras
ax.plot(X2D[:, 0], X2D[:, 1], "k.")  # "k." = puntos negros

# Etiquetas de los ejes: z1 y z2 representan las nuevas dimensiones
ax.set_xlabel("$z_1$", fontsize=18)
ax.set_ylabel("$z_2$", fontsize=18, rotation=0)

# Configuramos límites y grid
ax.axis([-1.5, 1.3, -1.2, 1.2])
ax.grid(True)

In [ ]:
# ============================================
# PCA CON SKLEARN - MÉTODO SIMPLIFICADO
# ============================================
# Ahora usamos la clase PCA de sklearn que hace todo automáticamente

from sklearn.decomposition import PCA

# Creamos el objeto PCA especificando cuántos componentes queremos
pca = PCA(n_components = 2)

# fit_transform hace dos cosas:
# 1. fit: calcula los componentes principales y centra los datos
# 2. transform: proyecta los datos en el nuevo espacio 2D
X2D = pca.fit_transform(X)

In [ ]:
# ============================================
# COMPONENTES PRINCIPALES CALCULADOS POR SKLEARN
# ============================================
# Verificamos que sklearn obtiene los mismos componentes que nuestro cálculo manual

# pca.components_ contiene los componentes principales en las FILAS
# Cada fila es un vector unitario que define una dirección de máxima varianza
pca.components_

In [ ]:
# ============================================
# VARIANZA EXPLICADA POR CADA COMPONENTE
# ============================================
# Nos dice qué porcentaje de la varianza total captura cada componente

# El primer componente captura ~84% de la varianza
# El segundo componente captura ~15% de la varianza
# INTERPRETACIÓN: los 2 primeros componentes capturan ~99% de la información
pca.explained_variance_ratio_

In [ ]:
# ============================================
# VARIANZA PERDIDA EN LA REDUCCIÓN
# ============================================
# Calculamos cuánta información perdemos al eliminar el 3er componente

# 1 - suma de varianzas explicadas = varianza perdida
# ~1.1% de la varianza se pierde (la del 3er componente)
# Esto confirma que la 3era dimensión era redundante
1 - pca.explained_variance_ratio_.sum()

In [ ]:
# ============================================
# VISUALIZACIÓN DE LOS RESULTADOS DE PCA (SKLEARN)
# ============================================
# Graficamos los datos proyectados usando el resultado de sklearn

fig = plt.figure()
ax = fig.add_subplot(111, aspect='equal')

# Notamos que la visualización es similar a la manual
# (puede tener signos invertidos, pero la estructura es la misma)
ax.plot(X2D[:, 0], X2D[:, 1], "k+")
ax.plot(X2D[:, 0], X2D[:, 1], "k.")

ax.set_xlabel("$z_1$", fontsize=18)
ax.set_ylabel("$z_2$", fontsize=18, rotation=0)
ax.axis([-1.3, 1.5, -1.2, 1.2])
ax.grid(True)

In [ ]:
# ============================================
# TRANSFORMACIÓN INVERSA: DE 2D DE VUELTA A 3D
# ============================================
# Intentamos reconstruir los datos originales a partir de los datos reducidos

# inverse_transform proyecta de vuelta al espacio original (3D)
# Pero la información del 3er componente se perdió, así que no será exacto
X3D_inv = pca.inverse_transform(X2D)

# Visualizamos los datos reconstruidos en 3D
fig = plt.figure(figsize=(6, 3.8))
ax = fig.add_subplot(111, projection='3d')

# Los puntos reconstruidos deberían estar cerca de los originales
ax.plot(X3D_inv[:, 0], X3D_inv[:, 1], X3D_inv[:, 2], "bo")

ax.set_xlim([-1.5, 1.3])
ax.set_ylim([-1.2, 1.2])
ax.set_zlim([-1, 1])
plt.show()

In [ ]:
# ============================================
# VERIFICACIÓN: ¿SON IGUALES LOS DATOS RECONSTRUIDOS?
# ============================================
# Comprobamos si la reconstrucción es perfecta

# np.allclose verifica igualdad con tolerancia numérica
# Resultado: False - los datos NO son exactamente iguales
# Esto es esperado: perdimos el ~1.1% de varianza
np.allclose(X3D_inv, X)

In [ ]:
# ============================================
# ERROR CUADRÁTICO MEDIO (MSE) DE LA RECONSTRUCCIÓN
# ============================================
# Medimos el error promedio por punto

# Calculamos la distancia euclidiana al cuadrado para cada punto
# y promediamos sobre todos los puntos
# MSE ≈ 0.01 - error muy pequeño, la reconstrucción es buena
np.mean(np.sum(np.square(X3D_inv - X), axis=1)) # mse

Para elegir el número correcto de dimensiones, podemos especificar un valor mínimo deseado para la varianza y mantener las $d$ dimensiones que cumplan con este criterio (para la visualización de datos, debemos establecer $d$ en 2 o 3).

In [ ]:
# ============================================
# ELECCIÓN AUTOMÁTICA DE DIMENSIONES POR VARIANZA
# ============================================
# Encontramos cuántos componentes necesitamos para capturar 95% de la varianza

# Calculamos la varianza explicada acumulada
cumsum = np.cumsum(pca.explained_variance_ratio_)

# np.argmax encuentra el primer índice donde cumsum >= 0.95
# +1 porque los índices empiezan en 0
d = np.argmax(cumsum >= 0.95) + 1

# Resultado: con 2 componentes ya capturamos >95% de la varianza
d

In [ ]:
# ============================================
# CARGA DEL DATASET MNIST
# ============================================
# MNIST: 70,000 imágenes de dígitos escritos a mano (28x28 píxeles)

from sklearn.datasets import fetch_openml

# Descargamos el dataset (puede tardar la primera vez)
mnist = fetch_openml('mnist_784', version=1)

# Convertimos las etiquetas a enteros sin signo (0-9)
mnist.target = mnist.target.astype(np.uint8)

In [ ]:
# ============================================
# DIVISIÓN EN TRAIN Y TEST
# ============================================
# Separamos los datos para entrenar y evaluar modelos

from sklearn.model_selection import train_test_split

# X: imágenes aplanadas (cada imagen 28x28 = 784 características)
# y: etiquetas (0-9)
X = mnist["data"].values
y = mnist["target"].values

# Dividimos en 75% train, 25% test (por defecto)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
# ============================================
# TAMAÑO DEL DATASET EN BYTES
# ============================================
# Vemos cuánta memoria ocupa el dataset completo en RAM

# nbytes: número de bytes que ocupa el array X en memoria
# ~439 MB para 70,000 imágenes
X.nbytes

In [ ]:
# ============================================
# DIMENSIONES DEL DATASET
# ============================================
# 70,000 muestras × 784 características (28×28 píxeles)

X.shape

In [ ]:
# ============================================
# CÁLCULO MANUAL DEL TAMAÑO EN MEGABYTES
# ============================================
# Verificamos el cálculo del tamaño en MB

# 70,000 imágenes × 8 bytes (float64) × 784 características
# Dividido por 1024 dos veces para convertir de bytes a MB
# Resultado: ~418.7 MB
70000*8*784 / 1024 / 1024 # en megabytes

In [ ]:
# ============================================
# DETERMINAR DIMENSIONES ÓPTIMAS PARA MNIST
# ============================================
# ¿Cuántos componentes necesitamos para capturar 95% de varianza?

# Creamos PCA sin especificar n_components para calcular TODOS
pca = PCA()
pca.fit(X_train)

# Calculamos varianza explicada acumulada
cumsum = np.cumsum(pca.explained_variance_ratio_)

# Encontramos el número de componentes necesario para 95%
d = np.argmax(cumsum >= 0.95) + 1

# Resultado: 153 componentes (de 784 originales)
# ¡Reducción de ~80% en dimensionalidad manteniendo 95% de información!
d

In [ ]:
# ============================================
# GRÁFICA DE VARIANZA EXPLICADA ACUMULADA
# ============================================
# Visualizamos cómo aumenta la varianza explicada con cada componente

plt.figure(figsize=(6,4))

# Graficamos la curva de varianza acumulada
plt.plot(cumsum, linewidth=3)

# Configuramos ejes
plt.axis([0, 784, 0, 1])
plt.xlabel("Dimensions")
plt.ylabel("Explained Variance")

# Marcamos el punto donde llegamos a 95%
plt.plot([d, d], [0, 0.95], "k:")  # línea vertical
plt.plot([0, d], [0.95, 0.95], "k:")  # línea horizontal
plt.plot(d, 0.95, "ko")  # punto de intersección

# La curva muestra un "codo": después de ~150 componentes, 
# los componentes adicionales aportan muy poca información
plt.grid(True)
plt.show()

In [ ]:
# ============================================
# PCA CON UMBRAL DE VARIANZA AUTOMÁTICO
# ============================================
# Sklearn puede determinar automáticamente el número de componentes

# n_components=0.95 significa: "elige el número mínimo de componentes
# que expliquen al menos el 95% de la varianza"
pca = PCA(n_components=0.95)

# Aplicamos PCA a los datos de entrenamiento
X_reduced = pca.fit_transform(X_train)

In [ ]:
# ============================================
# NÚMERO DE COMPONENTES SELECCIONADOS
# ============================================
# Verificamos cuántos componentes eligió sklearn

# n_components_ (con guión bajo): número de componentes después del fit
# Resultado: 153 (igual que nuestro cálculo manual)
pca.n_components_

In [ ]:
# ============================================
# VERIFICACIÓN DE LA VARIANZA EXPLICADA TOTAL
# ============================================
# Confirmamos que realmente capturamos ~95% de la varianza

# Sumamos la varianza explicada por los 153 componentes
# Resultado: 0.950 (95.0%)
np.sum(pca.explained_variance_ratio_)

In [ ]:
# ============================================
# DESCOMPRESIÓN: RECONSTRUCCIÓN DE LAS IMÁGENES
# ============================================
# Intentamos recuperar las imágenes originales desde los 153 componentes

# inverse_transform proyecta de vuelta al espacio original (784 dimensiones)
# Las imágenes reconstruidas serán similares pero no idénticas
X_recovered = pca.inverse_transform(X_reduced)

In [ ]:
# ============================================
# FUNCIÓN AUXILIAR PARA VISUALIZAR DÍGITOS MNIST
# ============================================
# Esta función dibuja múltiples imágenes de dígitos en una grilla

import matplotlib as mpl 

def plot_digits(instances, images_per_row=5, **options):
    size = 28  # cada imagen es 28×28 píxeles
    images_per_row = min(len(instances), images_per_row)
    
    # Convertimos cada vector de 784 elementos a una matriz 28×28
    images = [instance.reshape(size,size) for instance in instances]
    
    # Calculamos cuántas filas necesitamos
    n_rows = (len(instances) - 1) // images_per_row + 1
    
    # Rellenamos con imágenes vacías si es necesario
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    
    # Concatenamos imágenes horizontalmente por fila
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    
    # Concatenamos todas las filas verticalmente
    image = np.concatenate(row_images, axis=0)
    
    # Mostramos en escala de grises
    plt.imshow(image, cmap = mpl.cm.binary, **options)
    plt.axis("off")

In [ ]:
# ============================================
# COMPARACIÓN VISUAL: ORIGINALES VS COMPRIMIDOS
# ============================================
# Mostramos dígitos originales al lado de los reconstruidos

plt.figure(figsize=(7, 4))

# Panel izquierdo: imágenes originales
plt.subplot(121)
plot_digits(X_train[::2100])  # tomamos cada 2100-ésima imagen
plt.title("Original", fontsize=16)

# Panel derecho: imágenes reconstruidas desde 153 componentes
plt.subplot(122)
plot_digits(X_recovered[::2100])
plt.title("Comprimido", fontsize=16)

# Observación: las imágenes comprimidas son muy similares a las originales
# Perdimos solo 5% de información pero las imágenes siguen siendo reconocibles
plt.tight_layout()

Si el conjunto de datos es demasiado grande para caber en la memoria, podemos usar PCA incremental.

In [ ]:
# ============================================
# PCA INCREMENTAL PARA DATASETS GRANDES
# ============================================
# Cuando los datos no caben en memoria, procesamos por lotes (batches)

from sklearn.decomposition import IncrementalPCA
from tqdm import tqdm  # barra de progreso

n_batches = 100  # dividimos los datos en 100 lotes

# IncrementalPCA puede aprender de forma incremental (lote por lote)
inc_pca = IncrementalPCA(n_components=154)

# Procesamos cada lote por separado
for X_batch in tqdm(np.array_split(X_train, n_batches)):
    # partial_fit actualiza el modelo con un nuevo lote
    # Sin cargar todos los datos en memoria a la vez
    inc_pca.partial_fit(X_batch)

# Ahora transformamos todos los datos de entrenamiento
X_reduced = inc_pca.transform(X_train)

In [ ]:
# ============================================
# RECONSTRUCCIÓN CON INCREMENTAL PCA
# ============================================
# Proyectamos de vuelta a 784 dimensiones

# La reconstrucción con IncrementalPCA debería ser muy similar
# a la de PCA estándar
X_recovered_inc_pca = inc_pca.inverse_transform(X_reduced)

In [ ]:
# ============================================
# VARIANZA EXPLICADA CON INCREMENTAL PCA
# ============================================
# Verificamos la calidad del resultado

# Resultado: ~94.98% de varianza explicada
# Muy similar al PCA estándar (~95.0%)
np.sum(inc_pca.explained_variance_ratio_)

In [ ]:
# ============================================
# COMPARACIÓN VISUAL: INCREMENTAL PCA
# ============================================
# Las imágenes reconstruidas deberían verse casi idénticas

plt.figure(figsize=(7, 4))

# Izquierda: originales
plt.subplot(121)
plot_digits(X_train[::2100])

# Derecha: reconstruidas con IncrementalPCA
plt.subplot(122)
plot_digits(X_recovered_inc_pca[::2100])

# Observación: los resultados son prácticamente indistinguibles
# del PCA estándar, pero IncrementalPCA usa menos memoria
plt.tight_layout()

Podemos aplicar el *truco del kernel* para realizar proyecciones no lineales complejas para la reducción de dimensionalidad. Es bueno para preservar grupos de instancias después de la proyección e incluso desenrollar conjuntos de datos que se encuentran en un *manifold*.


In [ ]:
# ============================================
# DATASET SWISS ROLL (ROLLO SUIZO)
# ============================================
# Datos 3D con estructura NO LINEAL (manifold curvo)

from sklearn.datasets import make_swiss_roll

# Generamos 1000 puntos en forma de "rollo suizo"
# Este dataset es un ejemplo clásico de manifold no lineal
X, t = make_swiss_roll(n_samples=1000, noise=0.2, random_state=42)

# t contiene el "tiempo" o posición a lo largo del rollo (útil para colorear)

# Configuración de ejes para visualización
axes = [-11.5, 14, -2, 23, -12, 15]

# Visualizamos en 3D
fig = plt.figure(figsize=(6, 5))
ax = fig.add_subplot(111, projection='3d')

# Coloreamos los puntos según su posición en el rollo (t)
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=t, cmap=plt.cm.hot)
ax.view_init(10, -70)  # ángulo de vista

# Etiquetas de ejes
ax.set_xlabel("$x_1$", fontsize=18)
ax.set_ylabel("$x_2$", fontsize=18)
ax.set_zlabel("$x_3$", fontsize=18)

ax.set_xlim(axes[0:2])
ax.set_ylim(axes[2:4])
ax.set_zlim(axes[4:6])

plt.show()

In [ ]:
# ============================================
# KERNEL PCA - PCA NO LINEAL
# ============================================
# Comparamos diferentes kernels para desenrollar el swiss roll

from sklearn.decomposition import KernelPCA

# Tres tipos de kernels:

# 1. Kernel lineal: equivalente a PCA estándar
lin_pca = KernelPCA(n_components = 2, kernel="linear", fit_inverse_transform=True)

# 2. Kernel RBF (Radial Basis Function): captura relaciones no lineales
#    gamma controla el "radio" de influencia de cada punto
rbf_pca = KernelPCA(n_components = 2, kernel="rbf", gamma=0.0433, fit_inverse_transform=True)

# 3. Kernel sigmoide: similar a función de activación de redes neuronales
#    gamma y coef0 son hiperparámetros
sig_pca = KernelPCA(n_components = 2, kernel="sigmoid", gamma=0.001, coef0=1, fit_inverse_transform=True)

# Variable para clasificación binaria (dividir el rollo en dos)
y = t > 6.9

# Comparamos los tres kernels
plt.figure(figsize=(11, 4))

for subplot, pca, title in ((131, lin_pca, "Linear kernel"), 
                            (132, rbf_pca, "RBF kernel, $\gamma=0.04$"), 
                            (133, sig_pca, "Sigmoid kernel, $\gamma=10^{-3}, r=1$")):
    # Aplicamos PCA con cada kernel
    X_reduced = pca.fit_transform(X)
    
    # Guardamos el resultado RBF para uso posterior
    if subplot == 132:
        X_reduced_rbf = X_reduced
    
    # Graficamos
    plt.subplot(subplot)
    plt.title(title, fontsize=14)
    plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=t, cmap=plt.cm.hot)
    plt.xlabel("$z_1$", fontsize=18)
    if subplot == 131:
        plt.ylabel("$z_2$", fontsize=18, rotation=0)
    plt.grid(True)

# OBSERVACIÓN CLAVE:
# - Kernel lineal NO desenrolla el rollo (falla con datos no lineales)
# - Kernel RBF desenrolla el rollo exitosamente (preserva estructura)
# - Kernel sigmoide también ayuda pero menos que RBF
plt.show()

Seleccionar el mejor kernel y los valores de hiperparámetros no es una tarea seniclla, ya que PCA es una tarea no supervisada y no tenemos una métrica de rendimiento. Sin embargo, la reducción de dimensionalidad es normalmente un paso de preprocesamiento para una tarea de aprendizaje supervisado (por ejemplo, clasificación). Esto significa que podemos usar la búsqueda en cuadrícula para seleccionar los hiperparámetros óptimos que conduzcan al mejor rendimiento en esa tarea.

In [ ]:
# ============================================
# GRID SEARCH PARA OPTIMIZAR HIPERPARÁMETROS DE KERNEL PCA
# ============================================
# Encontramos el mejor kernel y gamma usando validación cruzada

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Creamos un pipeline: KernelPCA + Regresión Logística
# Idea: encontrar los hiperparámetros de KPCA que mejor funcionan
# para clasificar los datos después de la reducción
clf = Pipeline([
        ("kpca", KernelPCA(n_components=2)),
        ("log_reg", LogisticRegression(solver="lbfgs"))
    ])

# Definimos la grilla de hiperparámetros a probar
param_grid = [{
        "kpca__gamma": np.linspace(0.03, 0.05, 10),  # 10 valores entre 0.03 y 0.05
        "kpca__kernel": ["rbf", "sigmoid"]  # probamos 2 kernels
    }]

# GridSearchCV prueba todas las combinaciones (10 × 2 = 20)
# con validación cruzada de 3 folds
grid_search = GridSearchCV(clf, param_grid, cv=3)

# Entrenamos y buscamos los mejores hiperparámetros
# usando la tarea de clasificación (y) como criterio
grid_search.fit(X, y)

In [ ]:
# ============================================
# MEJORES HIPERPARÁMETROS ENCONTRADOS
# ============================================
# Mostramos la combinación óptima

# Resultado: kernel RBF con gamma ≈ 0.0433
# Es el que mejor preserva la estructura para la clasificación
print(grid_search.best_params_)

In [ ]:
# ============================================
# RECONSTRUCCIÓN CON KERNEL PCA (PREIMAGEN)
# ============================================
# Proyectamos de vuelta al espacio 3D original

# Usamos los mejores parámetros encontrados
rbf_pca = KernelPCA(n_components = 2, kernel="rbf", gamma=0.0433,
                    fit_inverse_transform=True)

# Reducimos a 2D
X_reduced = rbf_pca.fit_transform(X)

# Intentamos reconstruir el 3D original (preimagen)
# NOTA: con kernels no lineales, la reconstrucción es aproximada
X_preimage = rbf_pca.inverse_transform(X_reduced)

In [ ]:
# ============================================
# VISUALIZACIÓN DE LA PREIMAGEN
# ============================================
# Mostramos los datos reconstruidos en 3D

axes = [-11.5, 14, -2, 23, -12, 15]

fig = plt.figure(figsize=(6, 5))
ax = fig.add_subplot(111, projection='3d')

# Graficamos la preimagen (reconstrucción)
ax.scatter(X_preimage[:, 0], X_preimage[:, 1], X_preimage[:, 2], 
           c=t, cmap=plt.cm.hot)
ax.view_init(10, -70)

ax.set_xlabel("$x_1$", fontsize=18)
ax.set_ylabel("$x_2$", fontsize=18)
ax.set_zlabel("$x_3$", fontsize=18)

# OBSERVACIÓN: la preimagen NO es exacta (se ve diferente al original)
# La reconstrucción desde espacio no lineal pierde información
plt.show()

In [ ]:
# ============================================
# ERROR DE RECONSTRUCCIÓN (MSE)
# ============================================
# Medimos qué tan diferente es la preimagen del original

from sklearn.metrics import mean_squared_error

# MSE ≈ 32.79 - error considerable
# Esto confirma que la reconstrucción desde kernel no lineal
# no es perfecta (a diferencia de PCA lineal)
mean_squared_error(X, X_preimage)

## Otras técnicas

In [ ]:
# ============================================
# NUEVO DATASET SWISS ROLL
# ============================================
# Generamos un nuevo rollo suizo para comparar otras técnicas

# Usamos una semilla diferente (41) para variedad
X, t = make_swiss_roll(n_samples=1000, noise=0.2, random_state=41)

In [ ]:
# ============================================
# LLE (LOCALLY LINEAR EMBEDDING)
# ============================================
# Técnica que preserva relaciones locales entre vecinos

from sklearn.manifold import LocallyLinearEmbedding

# Parámetros:
# - n_components=2: reducir a 2 dimensiones
# - n_neighbors=10: cada punto considera sus 10 vecinos más cercanos
lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10, random_state=42)

# Aplicamos LLE
# LLE reconstruye cada punto como combinación lineal de sus vecinos
# y busca una proyección 2D que preserve estas relaciones
X_reduced = lle.fit_transform(X)

LLE funciona midiendo primero cómo cada instancia de entrenamiento se relaciona linealmente con sus vecinos más cercanos y luego buscando una representación de baja dimensión del conjunto de entrenamiento donde se preserven mejor estas relaciones locales.

In [ ]:
# ============================================
# VISUALIZACIÓN: LLE DESENROLLA EL SWISS ROLL
# ============================================
# LLE es excelente para desenrollar manifolds

plt.title("Unrolled swiss roll using LLE", fontsize=14)

# Graficamos los datos proyectados coloreados por t
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=t, cmap=plt.cm.hot)

plt.xlabel("$z_1$", fontsize=18)
plt.ylabel("$z_2$", fontsize=18)
plt.axis([-0.065, 0.055, -0.1, 0.12])
plt.grid(True)

# OBSERVACIÓN: LLE desenrolla perfectamente el rollo
# Preserva la estructura local (puntos cercanos siguen cercanos)
plt.show()

LLE funciona muy bien para desenrollar *manifolds*, pero escala mal a conjuntos de datos muy grandes.

In [ ]:
# ============================================
# MDS (MULTIDIMENSIONAL SCALING)
# ============================================
# Preserva las distancias entre TODOS los puntos (no solo vecinos)

from sklearn.manifold import MDS

# MDS intenta mantener las distancias par a par del espacio original
# Es más costoso computacionalmente que LLE
mds = MDS(n_components=2, random_state=42)
X_reduced_mds = mds.fit_transform(X)

In [ ]:
# ============================================
# ISOMAP (ISOMETRIC MAPPING)
# ============================================
# Similar a MDS pero usa distancias geodésicas (sobre el manifold)

from sklearn.manifold import Isomap

# Isomap construye un grafo de vecinos y calcula distancias
# a lo largo del manifold (no distancias euclidianas directas)
# Luego aplica MDS sobre estas distancias geodésicas
isomap = Isomap(n_components=2)
X_reduced_isomap = isomap.fit_transform(X)

In [ ]:
# ============================================
# t-SNE (T-DISTRIBUTED STOCHASTIC NEIGHBOR EMBEDDING)
# ============================================
# Técnica muy popular para visualización, preserva grupos/clusters

from sklearn.manifold import TSNE

# t-SNE es excelente para visualización pero:
# - Muy lento para datasets grandes
# - No garantiza preservar distancias globales
# - Preserva muy bien la estructura de clusters
tsne = TSNE(n_components=2, random_state=42)
X_reduced_tsne = tsne.fit_transform(X)

In [ ]:
# ============================================
# COMPARACIÓN DE LAS 3 TÉCNICAS
# ============================================
# Visualizamos MDS, Isomap y t-SNE lado a lado

titles = ["MDS", "Isomap", "t-SNE"]

plt.figure(figsize=(11,4))

for subplot, title, X_reduced in zip((131, 132, 133), titles,
                                     (X_reduced_mds, X_reduced_isomap, X_reduced_tsne)):
    plt.subplot(subplot)
    plt.title(title, fontsize=14)
    
    # Graficamos cada resultado coloreado por t
    plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=t, cmap=plt.cm.hot)
    plt.xlabel("$z_1$", fontsize=18)
    if subplot == 131:
        plt.ylabel("$z_2$", fontsize=18, rotation=0)
    plt.grid(True)

# COMPARACIÓN:
# - MDS: preserva distancias globales, desenrolla parcialmente
# - Isomap: muy similar a LLE, desenrolla bien usando distancias geodésicas  
# - t-SNE: agrupa bien pero distorsiona distancias, menos "desenrollado"

plt.show()